In [21]:
%reload_ext autoreload
%load_ext autoreload
%autoreload 2


The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [22]:
import PyPDF2

with open("ML Notes.pdf", "rb") as pdf_file:
    reader = PyPDF2.PdfReader(pdf_file)

# page = data.pages[page_num]
# page_image = Image.open(io.BytesIO(page.images[0].data)) # Assumes one image per page, use other logic if needed
# extracted_text += pytesseract.image_to_string(page_image, lang=lang)

In [23]:
import fitz
from PIL import Image
import io
pdf_document = fitz.open(pdf_file)

images = []
# Loop through each page of the PDF
for page_num in range(pdf_document.page_count):
    page = pdf_document.load_page(page_num)
    # Convert the page to an image
    pix = page.get_pixmap()
    img = Image.open(io.BytesIO(pix.tobytes()))  # Convert bytes to image
    images.append(img)

In [24]:
import cv2
import matplotlib.pyplot as plt
import numpy as np

def getArea(box):
    return (box[2] - box[0]) * (box[3] - box[1])


img = np.array(images[0])
new_height,new_width = int(img.shape[0])*3, int(img.shape[1])*3
img = cv2.resize(img, (new_width, new_height), interpolation=cv2.INTER_LANCZOS4)
height,width = int(img.shape[0]), int(img.shape[1])

# img = img[:int(h/4), :w, :]

img_gray = cv2.cvtColor(img, cv2.COLOR_RGB2GRAY)
ret, thresh2 = cv2.threshold(img_gray, 150, 255, cv2.THRESH_BINARY_INV)
kernel = cv2.getStructuringElement(cv2.MORPH_RECT, (150,2))
mask = cv2.morphologyEx(thresh2, cv2.MORPH_DILATE, kernel)

bboxes = []
bboxes_img = img.copy()
contours = cv2.findContours(mask, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
contours = contours[0] if len(contours) == 2 else contours[1]
cropped_images = []
areas = []

for cntr in contours:
    x,y,w,h = cv2.boundingRect(cntr)
    cv2.rectangle(bboxes_img, (x, y), (x+w, y+h), (0,0,255), 1)
    if(h > 10):
        cropped_images.append(img[max(0, y-15):min(y+h+15,height) , x: x+w, :])
        areas.append(h*w)
        bboxes.append((x,y,w,h))
    
print(len(cropped_images), len(areas))

unsharp_images = []
deblurred_unsharp_images = []
for idx, cropped_image in enumerate(cropped_images):

    # Apply Gaussian Blur
    blurred = cv2.GaussianBlur(cropped_image, (9, 9), 10)

    # Create an unsharp mask
    unsharp_image = cv2.addWeighted(cropped_image, 1.5, blurred, -0.5, 0)
    unsharp_images.append(unsharp_image)

    # plt.imshow(unsharp_image)
    # plt.show()

    kernel = np.array([[0, -1, 0], 
                   [-1, 5, -1], 
                   [0, -1, 0]])

    # Apply the kernel
    deblurred_image = cv2.filter2D(cropped_image, -1, kernel)
    deblurred_unsharp_image = cv2.filter2D(unsharp_image, -1, kernel)
    deblurred_unsharp_images.append(deblurred_unsharp_image)
    # cv2.imwrite(f"unsharp_{idx}.png", unsharp_image)
    # cv2.imwrite(f"deblurred_{idx}.png", deblurred_image)
    # cv2.imwrite(f"deblurred_unsharp_{idx}.png", deblurred_unsharp_image)
    # cv2.imwrite(f"original_{idx}.png", cropped_image)

    # plt.imshow(deblurred_image)
    # plt.show()

25 25


In [25]:
from transformers import TrOCRProcessor, VisionEncoderDecoderModel
from PIL import Image
import requests
from matplotlib import cm

processor = TrOCRProcessor.from_pretrained('microsoft/trocr-base-handwritten')
model = VisionEncoderDecoderModel.from_pretrained('microsoft/trocr-base-handwritten')

Config of the encoder: <class 'transformers.models.vit.modeling_vit.ViTModel'> is overwritten by shared encoder config: ViTConfig {
  "attention_probs_dropout_prob": 0.0,
  "encoder_stride": 16,
  "hidden_act": "gelu",
  "hidden_dropout_prob": 0.0,
  "hidden_size": 768,
  "image_size": 384,
  "initializer_range": 0.02,
  "intermediate_size": 3072,
  "layer_norm_eps": 1e-12,
  "model_type": "vit",
  "num_attention_heads": 12,
  "num_channels": 3,
  "num_hidden_layers": 12,
  "patch_size": 16,
  "qkv_bias": false,
  "transformers_version": "4.47.1"
}

Config of the decoder: <class 'transformers.models.trocr.modeling_trocr.TrOCRForCausalLM'> is overwritten by shared decoder config: TrOCRConfig {
  "activation_dropout": 0.0,
  "activation_function": "gelu",
  "add_cross_attention": true,
  "attention_dropout": 0.0,
  "bos_token_id": 0,
  "classifier_dropout": 0.0,
  "cross_attention_hidden_size": 768,
  "d_model": 1024,
  "decoder_attention_heads": 16,
  "decoder_ffn_dim": 4096,
  "decoder

In [28]:

text = []

for cropped_image in unsharp_images:

    image = Image.fromarray(cropped_image, 'RGB')
    pixel_values = processor(images=image, return_tensors="pt").pixel_values
    generated_ids = model.generate(pixel_values)
    generated_text = processor.batch_decode(generated_ids, skip_special_tokens=True)[0]
    text.append(generated_text)


text = text[::-1]


In [30]:
text

['# 2nd.laoso',
 'Machine learning .',
 '# To make a machine ! learn from past experience ! and',
 'impose , the performance of intelligent programs ...',
 '# Definition : A companion program is " said to learn from experiment ,',
 'with respect to task # and performance missions # it',
 'in performance at task in its all machine by R. improves',
 'with experience E. ( Mitchell definition ) .',
 '# AIGORMINOUTH , S.MIN. Precision Free , etc ... .',
 'Cancer Diagnosis .',
 'Task T. charity malignant " benign " rumors . "',
 'performance Measure " p. " " " No. of cancer patients correctly ,',
 'obing passed ... .',
 'Training experience , It - a database of cancer and non-oms .',
 'spallients .',
 '2 . This Dataset :',
 '2 As a kind of effloureux . "',
 'by given the dataset , over , Mr. algorithm , should be able',
 'to predict the type of the flowers ... .',
 'displaystyle',
 'LS Fahlins , - , Sepal , lengths , with # petal length , width ,',
 '0 0',
 '1955 Virginia Senator',
 'It give

In [38]:
import google.generativeai as genai
import os

genai.configure(api_key=os.getenv("google_gemini_api_key"))
llm_model = genai.GenerativeModel("gemini-1.5-flash")

output = llm_model.generate_content(f"""I have a text read from ocr (performed on handwritten text) regarding machine learning notes taken in class. 
                                    But it is not very accurate. Give me the corrected version of the text without any explanations or assumptions.
                                    Make sure to use your own knowledge on the subject as well when making corrections.
                       ocr text: "{text}"
                       """).text


In [39]:
print(output)

Machine learning.

To make a machine learn from past experience and improve the performance of intelligent programs.

Definition: A computer program is said to learn from experience E with respect to some task T and performance measure P, if its performance at T, as measured by P, improves with experience E. (Mitchell definition).

Algorithms: SVM, MIN, Precision, Recall, etc.

Cancer Diagnosis:

Task T: Classify malignant "cancer" or benign "tumors".

Performance Measure P: Number of cancer patients correctly diagnosed.

Training Experience E: A database of cancer and non-cancer patients.

2.  This Dataset:

As a type of classifier.

Given a dataset, an algorithm should be able to predict the type of flowers.

Features: Sepal length, sepal width, petal length, petal width.

Iris dataset: Iris setosa (Mexico, California)



In [85]:
import os
os.environ["GOOGLE_APPLICATION_CREDENTIALS"] = "/Users/roobansappani/Desktop/EduAI/EduAI/gc_vision_creds.json"

In [87]:
from google.cloud import vision
import io
from PIL import Image


# Load the image using PIL
image = images[0]

# Convert the PIL image to a byte array
byte_arr = io.BytesIO()
image.save(byte_arr, format=image.format)
byte_arr = byte_arr.getvalue()

# Initialize the Vision API client
client = vision.ImageAnnotatorClient()

# Create a Google Cloud Vision Image object
gcv_image = vision.Image(content=byte_arr)


# Perform text detection
response = client.text_detection(image=gcv_image)
annotations = response.text_annotations

# Extract and print the detected text
if annotations:
    print("Detected text:")
    print(annotations[0].description)
else:
    print("No text detected.")

# Check for errors
if response.error.message:
    raise Exception(f"API error: {response.error.message}")


Detected text:
27/07/20
Machine
Learning
-
To make
a
machine
learn from past experiences
and
Definition: A computer program
improve the performance of intelligent programs.
is said to learn from experienceE
with respect
to
its
task T and perfor
performane at task in T, as
mance
we as une
by
measure f
improves
if
with
experience E. (Mitchell definition).
->>
Algorithms: KHN, SVM, Desicion Izee
etc.
Task T-
Cancer Diagnosis:
Classify malignant / benign
tumors
Performance Measure
P
-
"No.
of
cancer patients correctly
diagnosed...
Training
patienti
Iris Dataset:
experience E-
a database of
cancer and non-conay
↳ 3 kind
of flowers.
Ly
given
the
dataret, our
ML algorithm should
be able
to perdict the type of the flower.
↳ Features
-
Sepal length, width & petal length, width
4 types
-
Iris setosa 10) Versicolor (1), Virginica (2)
Scanned with CamScanner


In [89]:
import PyPDF2

with open("temp.pdf", "rb") as pdf_file:
    reader = PyPDF2.PdfReader(pdf_file)

    text = "".join(page.extract_text() for page in reader.pages)

In [125]:
print(text[-5000:])

pose of closing the 
accounts. 
This document has a strictly defined form. 
It should be performed in a reliable and comparable manner so that its 
addressee can efficiently read all the necessary information.
The financial statements shall consist of:
1) balance sheet;
(2)the profit and loss account;
(3)notes, including an introduction to the financial statements and 
additional information and explanations.Scope of the reportFinancial statements of the entity73The report shall be distinguished by:
Credibility (reliability),
Intelligibility for users,
Completeness
Comparability ,
Verifiability ,
Punctuality ,
Business continuity.
•Investors 
•creditors using reports when making decisions on granting a loan, bank 
loan or sale with a deferred payment date,
•government agencies and local governments, using reports when assessing 
the compliance of the company's activities with applicable legal 
regulations,
•competition using reports when assessing strengths, weaknesses and the 
company

In [121]:
text.count(" ")

6359

In [40]:
from google.cloud import vision

client = vision.ImageAnnotatorClient()


image = vision.Image(content=unsharp_images[0])

response = client.document_text_detection(image=image)

DefaultCredentialsError: Your default credentials were not found. To set up Application Default Credentials, see https://cloud.google.com/docs/authentication/external/set-up-adc for more information.

In [ ]:
response

In [91]:
import google.generativeai as genai
import os

genai.configure(api_key=os.getenv("google_gemini_api_key"))
model = genai.GenerativeModel("gemini-1.5-flash")

In [101]:
mock_test_question_template = """
You are an Education AI assistant tailored to make mock questions and answers for students so that they can prepare for their exam. 
You will be preparing these mock questions and answers based on the context given in the <context> tags.
Make sure to have a mix of MCQ, written and one word qnswer questions.

<context>
{context}
</context>

When you prepare the questions make sure to have it in the format given in the <question_format>.

<question>
1. question 1
2. question 2
.
.
</question>

First 3 questions are MCQ, 
Next 3 are one word answers
Next 4 are general.
Prepare questions in the given format based on the given context.

Qestions:

"""


In [99]:
final_prompt = mock_test_question_template.format(context = text)

response = model.generate_content(final_prompt)

In [100]:
print(response.text)

<question>
1.  What are the three components of the accounting system as outlined in the text?

2. Multiple Choice: The balance sheet method in accounting is based on which principle?
    a) Single-entry bookkeeping
    b) Double-entry bookkeeping
    c) Triple-entry bookkeeping
    d) None of the above

3. What is the subjective method in accounting, and provide an example illustrating its application?

4.  What is the basic legal act regulating the accounting system in most countries?

5. What are the three criteria that must be met for an asset to be included in the balance sheet?

6. Define 'equity' as it relates to accounting.

7.  What is the difference between an economic event and an economic operation?  Give an example of each.

8.  One-word answer: What type of accounting involves the anticipation of tax burdens and informing about current obligations to the tax office?

9.  What are the key functions of accounting books, differentiating between accounting books and tax books